In [2]:
from pathlib import Path
import pandas as pd
import pgeocode

In [3]:
users_path = Path.cwd() / 'data/sd254_users.csv'
cards_path = Path.cwd() / 'data/sd254_cards.csv'

users_df = pd.read_csv(users_path)
users_df['User'] = users_df.index
cards_df = pd.read_csv(cards_path)

In [4]:
display(users_df.head())
display(cards_df.head())

,Person,Current Age,Retirement Age,Birth Year,Birth Month,Gender,Address,Apartment,City,State,Zipcode,Latitude,Longitude,Per Capita Income - Zipcode,Yearly Income - Person,Total Debt,FICO Score,Num Credit Cards,User
0,Hazel Robinson,53,66,1966,11,Female,462 Rose Lane,NaN,La Verne,CA,91750,34.15,-117.76,$29278,$59696,$127613,787,5,0
1,Sasha Sadr,53,68,1966,12,Female,3606 Federal Boulevard,NaN,Little Neck,NY,11363,40.76,-73.74,$37891,$77254,$191349,701,5,1
2,Saanvi Lee,81,67,1938,11,Female,766 Third Drive,NaN,West Covina,CA,91792,34.02,-117.89,$22681,$33483,$196,698,5,2
3,Everlee Clark,63,63,1957,1,Female,3 Madison Street,NaN,New York,NY,10069,40.71,-73.99,$163145,$249925,$202328,722,4,3
4,Kyle Peterson,43,70,1976,9,Male,9620 Valley Stream Drive,NaN,San Francisco,CA,94117,37.76,-122.44,$53797,$109687,$183855,675,1,4


,User,CARD INDEX,Card Brand,Card Type,Card Number,Expires,CVV,Has Chip,Cards Issued,Credit Limit,Acct Open Date,Year PIN last Changed,Card on Dark Web
0,0,0,Visa,Debit,4344676511950444,12/2022,623,YES,2,$24295,09/2002,2008,No
1,0,1,Visa,Debit,4956965974959986,12/2020,393,YES,2,$21968,04/2014,2014,No
2,0,2,Visa,Debit,4582313478255491,02/2024,719,YES,2,$46414,07/2003,2004,No
3,0,3,Visa,Credit,4879494103069057,08/2024,693,NO,1,$12400,01/2003,2012,No
4,0,4,Mastercard,Debit (Prepaid),5722874738736011,03/2009,75,YES,1,$28,09/2008,2009,No


In [5]:
transactions_path = Path.cwd() / 'data/credit_card_transactions-ibm_v2.csv'

transactions_df = pd.read_csv(transactions_path)


In [6]:
transactions_df.loc[(transactions_df['User'] < 500, 'Is Fraud?')].value_counts()

Is Fraud?
No     5920547
Yes       6847
Name: count, dtype: int64

In [7]:
toy_transactions = transactions_df.loc[transactions_df['User'] < 100]
toy_transactions['Is Fraud?'].value_counts()

Is Fraud?
No     1256923
Yes       1310
Name: count, dtype: int64

In [8]:
merge_step_1 = toy_transactions.merge(users_df, how='inner', on='User')
merge_step_1.head()

,User,Card,Year,Month,Day,Time,Amount,Use Chip,Merchant Name,Merchant City,...,City,State,Zipcode,Latitude,Longitude,Per Capita Income - Zipcode,Yearly Income - Person,Total Debt,FICO Score,Num Credit Cards
0,0,0,2002,9,1,06:21,$134.09,Swipe Transaction,3527213246127876953,La Verne,...,La Verne,CA,91750,34.15,-117.76,$29278,$59696,$127613,787,5
1,0,0,2002,9,1,06:42,$38.48,Swipe Transaction,-727612092139916043,Monterey Park,...,La Verne,CA,91750,34.15,-117.76,$29278,$59696,$127613,787,5
2,0,0,2002,9,2,06:22,$120.34,Swipe Transaction,-727612092139916043,Monterey Park,...,La Verne,CA,91750,34.15,-117.76,$29278,$59696,$127613,787,5
3,0,0,2002,9,2,17:45,$128.95,Swipe Transaction,3414527459579106770,Monterey Park,...,La Verne,CA,91750,34.15,-117.76,$29278,$59696,$127613,787,5
4,0,0,2002,9,3,06:23,$104.71,Swipe Transaction,5817218446178736267,La Verne,...,La Verne,CA,91750,34.15,-117.76,$29278,$59696,$127613,787,5


In [9]:
merged_df = merge_step_1.merge(cards_df, how='inner', left_on=['User', 'Card'], right_on=['User', 'CARD INDEX'])
merged_df.head()

,User,Card,Year,Month,Day,Time,Amount,Use Chip,Merchant Name,Merchant City,...,Card Type,Card Number,Expires,CVV,Has Chip,Cards Issued,Credit Limit,Acct Open Date,Year PIN last Changed,Card on Dark Web
0,0,0,2002,9,1,06:21,$134.09,Swipe Transaction,3527213246127876953,La Verne,...,Debit,4344676511950444,12/2022,623,YES,2,$24295,09/2002,2008,No
1,0,0,2002,9,1,06:42,$38.48,Swipe Transaction,-727612092139916043,Monterey Park,...,Debit,4344676511950444,12/2022,623,YES,2,$24295,09/2002,2008,No
2,0,0,2002,9,2,06:22,$120.34,Swipe Transaction,-727612092139916043,Monterey Park,...,Debit,4344676511950444,12/2022,623,YES,2,$24295,09/2002,2008,No
3,0,0,2002,9,2,17:45,$128.95,Swipe Transaction,3414527459579106770,Monterey Park,...,Debit,4344676511950444,12/2022,623,YES,2,$24295,09/2002,2008,No
4,0,0,2002,9,3,06:23,$104.71,Swipe Transaction,5817218446178736267,La Verne,...,Debit,4344676511950444,12/2022,623,YES,2,$24295,09/2002,2008,No


In [15]:
# add column to define whether international
merged_df['International'] = merged_df['Merchant State'].str.len() > 2

# remove $ from columns
merged_df['Per Capita Income - Zipcode'] = merged_df['Per Capita Income - Zipcode'].str.replace('$', '').astype('int64')
merged_df['Yearly Income - Person'] = merged_df['Yearly Income - Person'].str.replace('$', '').astype('int64')
merged_df['Total Debt'] = merged_df['Total Debt'].str.replace('$', '').astype('int64')




                   

In [17]:
# create income-to-debt
merged_df['income_to_debt'] = merged_df['Yearly Income - Person'] / (merged_df['Total Debt'] + 0.001)

In [18]:
def calculate_distance(zip1, zip2):
    try:
        zip1 = str(int(zip1))
        zip2 = str(int(zip2))
    
        dist = pgeocode.GeoDistance('us')
        return dist.query_postal_code(zip1, zip2)
    except:
        return -1

In [20]:
merged_df['Distance'] = merged_df.apply(lambda row: calculate_distance(row['Zip'], row['Zipcode']), axis=1)

ValueError: cannot convert float NaN to integer

In [16]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1258233 entries, 0 to 1258232
Data columns (total 47 columns):
 #   Column                       Non-Null Count    Dtype  
---  ------                       --------------    -----  
 0   User                         1258233 non-null  int64  
 1   Card                         1258233 non-null  int64  
 2   Year                         1258233 non-null  int64  
 3   Month                        1258233 non-null  int64  
 4   Day                          1258233 non-null  int64  
 5   Time                         1258233 non-null  object 
 6   Amount                       1258233 non-null  object 
 7   Use Chip                     1258233 non-null  object 
 8   Merchant Name                1258233 non-null  int64  
 9   Merchant City                1258233 non-null  object 
 10  Merchant State               1075669 non-null  object 
 11  Zip                          1067603 non-null  float64
 12  MCC                          1258233 non-n

In [70]:
transactions_df.loc[transactions_df['Merchant State'].str.len() > 2, ['Merchant State', 'Merchant City', 'Zip']]

,Merchant State,Merchant City,Zip
407,Switzerland,Zurich,NaN
408,Switzerland,Zurich,NaN
906,Estonia,Tallinn,NaN
907,Estonia,Tallinn,NaN
908,Estonia,Tallinn,NaN
...,...,...,...
24385846,Argentina,Buenos Aires,NaN
24385847,Argentina,Buenos Aires,NaN
24385848,Argentina,Buenos Aires,NaN
24385849,Argentina,Buenos Aires,NaN


In [ ]:
transactions_df['Merchant State'].str.len() > 2

## Clean up Columns

Unsure how to deal with State or Zipcode. If we keep Zipcode, how do we deal with this as this number is a not a continuous number series?


An idea would be to calculate the distance between the user zipcode and the merchant zipcode?


### cards_df
Drop:
- Card Number
- Expires
- CVV
- Acct Open Date

### users_df
- Person
- Retirement Age
- Birth Year
- Gender
- Address
- Apartment
- City
- Latitude
- Longitude

### transactions_df
- Year
- Day
- Time
- Merchant State
  

In [9]:
transactions_df.head(5)

,User,Card,Year,Month,Day,Time,Amount,Use Chip,Merchant Name,Merchant City,Merchant State,Zip,MCC,Errors?,Is Fraud?
0,0,0,2002,9,1,06:21,$134.09,Swipe Transaction,3527213246127876953,La Verne,CA,91750.0,5300,NaN,No
1,0,0,2002,9,1,06:42,$38.48,Swipe Transaction,-727612092139916043,Monterey Park,CA,91754.0,5411,NaN,No
2,0,0,2002,9,2,06:22,$120.34,Swipe Transaction,-727612092139916043,Monterey Park,CA,91754.0,5411,NaN,No
3,0,0,2002,9,2,17:45,$128.95,Swipe Transaction,3414527459579106770,Monterey Park,CA,91754.0,5651,NaN,No
4,0,0,2002,9,3,06:23,$104.71,Swipe Transaction,5817218446178736267,La Verne,CA,91750.0,5912,NaN,No


In [71]:
transactions_df['MCC'].nunique()

109

In [37]:
cards_df = cards_df.drop(columns=['Card Number',
                       'Expires',
                       'CVV',
                       'Acct Open Date',
                       ], axis=1)

In [39]:
users_df.head()

,Person,Current Age,Retirement Age,Birth Year,Birth Month,Gender,Address,Apartment,City,State,Zipcode,Latitude,Longitude,Per Capita Income - Zipcode,Yearly Income - Person,Total Debt,FICO Score,Num Credit Cards,User
0,Hazel Robinson,53,66,1966,11,Female,462 Rose Lane,NaN,La Verne,CA,91750,34.15,-117.76,$29278,$59696,$127613,787,5,0
1,Sasha Sadr,53,68,1966,12,Female,3606 Federal Boulevard,NaN,Little Neck,NY,11363,40.76,-73.74,$37891,$77254,$191349,701,5,1
2,Saanvi Lee,81,67,1938,11,Female,766 Third Drive,NaN,West Covina,CA,91792,34.02,-117.89,$22681,$33483,$196,698,5,2
3,Everlee Clark,63,63,1957,1,Female,3 Madison Street,NaN,New York,NY,10069,40.71,-73.99,$163145,$249925,$202328,722,4,3
4,Kyle Peterson,43,70,1976,9,Male,9620 Valley Stream Drive,NaN,San Francisco,CA,94117,37.76,-122.44,$53797,$109687,$183855,675,1,4


In [28]:
transactions_df.loc[transactions_df['Year'] >= 2017].to_csv('data/test_set.csv')